# Building the Doublet Hash

In [ ]:
import anydbm, string, re
doubhash = anydbm.open(‘doub’, ‘n’)
literalhash = anydbm.open(‘literal’, ‘n’)
in_file = open(‘c:\\ftp\\neocl.xml’, “r”)
singular = re.compile(‘omas’)
england = re.compile(‘tumou?rs?’)
phrase = “”
for line in in_file:
    neoplasm_match = re.search(r’\”\> ?(.+) ?\<’, line)
    if neoplasm_match:
        phrase = neoplasm_match.group(1)
        phrase = singular.sub(“oma”,phrase)
        phrase = england.sub(“tumor”,phrase)
        literalhash[phrase] = “”
    hoparray = phrase.split()
    hoparray.append(“ “)
    for i in range(len(hoparray)-1):
        doublet = hoparray[i] + “ “ + hoparray[i + 1]
        if doubhash.has_key(doublet):
            continue
        doubhash_match = re.search(r’[a-z]+ [a-z]+’, doublet)
        if doubhash_match:
            doubhash[doublet] = “”
doubhash.close()
literalhash.close()

# Scanning the Literature for Candidate Terms

In [ ]:
import anydbm, string, re
doubhash = anydbm.open(‘doub’)
literalhash = anydbm.open(‘literal’)
newhash = {}
in_file = open(‘c:\\big\\cancer_gene_titles.txt’, ‘r’)
line = “ “
count = 0
singular = re.compile(‘omas’)
england = re.compile(‘tumou?rs?’)
for line in in_file:
    bigline = line.rstrip(“ \n”)
    bigline = singular.sub(“oma”, bigline)
    bigline = england.sub(“tumor”, bigline)
    englishline = “”
    hoparray = bigline.split()
    hoparray.append(“ “)
    for i in range(len(hoparray) - 1):
        doublet = hoparray[i] + “ “ + hoparray[i + 1]
        if doubhash.has_key(doublet):
            if (englishline != “”):
                englishline = englishline + “ “ + hoparray[i + 1]
            else:
                englishline = doublet
        else:
            if englishline != “”:
                englishline = englishline.strip()
                englishline = re.sub(r’^the ‘, “”, englishline)
                englishline = re.sub(r’^in ‘, “”, englishline)
                englishline = re.sub(r’^of ‘, “”, englishline)
                englishline = re.sub(r’^and ‘, “”, englishline)
                englishline = re.sub(r’^with ‘, “”, englishline)
                englishline = re.sub(r’^from ‘, “”, englishline)
                englishline = re.sub(r’^ a’, “”, englishline)
                englishline = re.sub(r’ the$’, “”, englishline)
                englishline = re.sub(r’ in$’, “”, englishline)
                englishline = re.sub(r’ of$’, “”, englishline)
                englishline = re.sub(r’ and$’, “”, englishline)
                englishline = re.sub(r’ with$’, “”, englishline)
                englishline = re.sub(r’ from$’, “”, englishline)
                englishline = re.sub(r’ a$’, “”, englishline)
                if literalhash.has_key(englishline):
                    continue
                if newhash.has_key(englishline):
                    continue
                phrase_match = re.search(r’ [a-z]+ ‘, englishline)
                if phrase_match:
                    count = count + 1
                    print str(count) + “ “ + englishline
                    newhash[englishline] = “”
doubhash.close()
literalhash.close()

# Adding Terms to the Neoplasm Classification

In [ ]:
import re, string
vocab_in = open(“c:\\ftp\\neocl.xml”, “r”)
doub_hash = {}
for line in vocab_in:
    code_match = re.search(r’C[0-9]{7}’, line)
    if not code_match:
        continue
    line_match = re.search(r’\”\> ?(.+) ?\<\/’, line)
    if line_match:
        phrase = line_match.group(1)
        doub_hash[phrase] = “”
vocab_in.close()
candidate_file = open(“c:\\ftp\\neocl.lst”, “r”)
out_file = open(“new.out”, “w”)
for line in candidate_file:
    line = re.sub(r’\n’,””, line)
    if (line == “”):
        continue
    if doub_hash.has_key(line):
        print line + “ already exists”
    else:
        print>>out_file, line

# Determining the Lineage of Every Neoplasm Concept

In [ ]:
import xml.parsers.expat
import re
parsefile = open(‘c:\\ftp\\neocl.xml’,’r’)
filestring = parsefile.read()
lastname = “”
code = “”
count = 0
text = “”
def start_element(name, attrs):
    global lastname
    global code
    if attrs.has_key(“nci-code”):
        code = attrs[“nci-code”]
    else:
        lastname = name + “>” + lastname
def end_element(name):
    global count
    global code
    global text
    global lastname
    if name == “name”:
        count = count + 1
        print str(count) + “|” + text + “|” + code + “|” + lastname + “\n”
        text = “”
    lastname = re.sub(name + r’>’,’’, lastname)
def char_data(data):
    global text
    text = repr(data)
    textmatch = re.search(r’\’(.+)\’’,text)
    if textmatch:
        text = textmatch.group(1)
p = xml.parsers.expat.ParserCreate()
p.StartElementHandler = start_element
p.EndElementHandler = end_element
p.CharacterDataHandler = char_data
p.Parse(filestring)